# Implementation of TextRank
(Based on: https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

The input text is given below

In [1]:
#Source of text:
#https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents

Text = "a b c d b c"

### Cleaning Text Data

The raw input text is cleaned off non-printable characters (if any) and turned into lower case.
The processed input text is then tokenized using NLTK library functions. 

In [4]:

import nltk
from nltk import word_tokenize
import string

#nltk.download('punkt')

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = list(filter(lambda x: x in printable, text)) #filter funny characters, if any.
    return "".join(text)

Cleaned_text = clean(Text)

text = word_tokenize(Cleaned_text)

print("Tokenized Text: \n")
print(text)

Tokenized Text: 

['a', 'b', 'c', 'd', 'b', 'c']


### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

Description of POS tags: 


http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [6]:
#nltk.download('averaged_perceptron_tagger')
  
POS_tag = nltk.pos_tag(text)

print("Tokenized Text with POS tags: \n")
print(POS_tag)

Tokenized Text with POS tags: 

[('a', 'DT'), ('b', 'NN'), ('c', 'NN'), ('d', 'NN'), ('b', 'NN'), ('c', 'NN')]


### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

Details about lemmatization: 
    
https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [8]:
#nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print("Text tokens after lemmatization of adjectives and nouns: \n")
print(lemmatized_text)

Text tokens after lemmatization of adjectives and nouns: 

['a', 'b', 'c', 'd', 'b', 'c']


## Vocabulary Creation

Vocabulary will only contain unique words from processed_text.

In [11]:
vocabulary = list(set(lemmatized_text))
print(vocabulary)

['b', 'c', 'a', 'd']


### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [30]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 2
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        print("i",vocabulary[i])
        print("j",vocabulary[j])
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(lemmatized_text)-window_size+1)):
                
                window_end = window_start+window_size
                
                window = lemmatized_text[window_start:window_end]
                print(window)
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        print("counted")
                        weighted_edge[i][j]+=1
                        covered_coocurrences.append([index_of_i,index_of_j])
        print("next loop\n")

print(vocabulary)
print("\nweighted matrix\n", weighted_edge)



i b
j b
next loop

i b
j c
['a', 'b']
['b', 'c']
counted
['c', 'd']
['d', 'b']
['b', 'c']
counted
next loop

i b
j a
['a', 'b']
counted
['b', 'c']
['c', 'd']
['d', 'b']
['b', 'c']
next loop

i b
j d
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b']
counted
['b', 'c']
next loop

i c
j b
['a', 'b']
['b', 'c']
counted
['c', 'd']
['d', 'b']
['b', 'c']
counted
next loop

i c
j c
next loop

i c
j a
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b']
['b', 'c']
next loop

i c
j d
['a', 'b']
['b', 'c']
['c', 'd']
counted
['d', 'b']
['b', 'c']
next loop

i a
j b
['a', 'b']
counted
['b', 'c']
['c', 'd']
['d', 'b']
['b', 'c']
next loop

i a
j c
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b']
['b', 'c']
next loop

i a
j a
next loop

i a
j d
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b']
['b', 'c']
next loop

i d
j b
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b']
counted
['b', 'c']
next loop

i d
j c
['a', 'b']
['b', 'c']
['c', 'd']
counted
['d', 'b']
['b', 'c']
next loop

i d
j a
['a', 'b']
['b', 'c']
['c', 'd']
['d', 'b